In [1]:
import pImpactR as impact
np=impact.np
plt=impact.plot.plt
from copy import deepcopy as copy

In [2]:
def compare(beam,lattice,elemList,line):
    impactM = impact.getTransferMap(beam,lattice,epsilon=[1e-9,1e-8,1e-9,1e-8,1e-5,1.0],order=3)
    ke = beam.kinetic_energy
    mass = beam.mass
    freq = beam.frequency

    g = ke/mass+1.0
    bg = np.sqrt(g**2-1.0)
    b = bg/g
    impactM.values[:5,-1]=-impactM.values[:5,-1]*mass*bg
    
    elemList1 = copy(elemList)
    mapout = impact.MLI.getElem.ptm()
    elemList1.append(mapout)
    Labor = impact.MLI.buildLabor(elemList1[-1:])
    Labor.list.insert(0,line.name)
    impact.MLI.writeInputfile(elemList1,line,Labor)
    impact.MLI.run()
    M,G = impact.MLI.readTransferMap()
    
    return impactM, M

# from Impact Input 

In [3]:
beam_impact,lattice_impact = impact.readInputFile('../../test.iota.sextupole.in')
# beam_impact,lattice_impact = impact.readInputFile('../../test.Chad.in')
beam_impact.nCore_y=1
beam_impact.nCore_z=1
beam_impact.n_particles = 1

ke = beam_impact.kinetic_energy
mass = beam_impact.mass
freq = beam_impact.frequency

g = ke/mass+1.0
bg = np.sqrt(g**2-1.0)
b = bg/g

reading ImpactZ input file (../../test.iota.sextupole.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
lattice_impact = impact.clearLattice(lattice_impact)

In [5]:
for elem in lattice_impact:
    if 'insert' in elem.type:
        elem.n_sckick=100
    else:
        if 'n_sckick' in elem:
            elem.n_sckick = int(np.ceil(elem.length*20))
            elem.n_map = 1

In [6]:
elemList_impact,latticeElem_impact = impact.impact2mli(beam_impact,lattice_impact)

Impact elem type RFkick is not recognized from MLI. skipping...


In [7]:
elemList_impact[2].lfrngquad = 1
elemList_impact[2].tfrngquad = 1
elemList_impact[2].driftexact = 1

In [8]:
L=0
for elem in elemList_impact:
    if 'l' in elem:
        L=L+elem['l']
print(L)

38.16822972000003


In [9]:
L=0
for elem in lattice_impact:
    if 'length' in elem:
        L=L+elem['length']
print(L)

39.96822972000005


### Sext off / NL on

In [10]:
M_impact,M_mli = compare(beam_impact,lattice_impact,elemList_impact,latticeElem_impact)

In [11]:
M_impact

,0,1,2,3,4,5
0,-0.699614,0.269226,2.306276e-09,0.000002,1.525068e-21,0.327065
1,-1.280303,-0.936673,-4.312842e-10,0.000002,7.716732e-22,0.246375
2,0.000000,0.000000,9.707320e-02,1.856125,0.000000e+00,-0.000000
3,0.000000,0.000000,-4.742467e-01,1.233490,0.000000e+00,-0.000000
4,-8.875665,-13.425935,1.245373e-07,-0.000078,1.000000e+00,-45.021149
5,-0.000000,-0.000000,-0.000000e+00,-0.000000,-4.454864e-13,1.000000


In [12]:
M_mli

,1,2,3,4,5,6
1,-0.699614,0.269226,0.000000,0.000000,0.0,0.327066
2,-1.280303,-0.936673,0.000000,0.000000,0.0,0.246376
3,0.000000,0.000000,0.097073,1.856125,0.0,0.000000
4,0.000000,0.000000,-0.474247,1.233490,0.0,0.000000
5,-0.246376,-0.372684,0.000000,0.000000,1.0,-1.249723
6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000


### Sext off / NL off

In [13]:
for item in lattice_impact:
    if 'insert' in item.type:
        item.strength_t = 0.0
        print(item)


             length: 1.8 [m]
           n_sckick: 100 [1]
              n_map: 10 [1]
         strength_t: 0.0
 transverse_scale_c: 0.01
        pipe_radius: 1.0 [m]
       tune_advance: 0.30000000043929054
               type: 'nonlinear_insert'


In [14]:
for item in elemList_impact:
    if 'insert' in item.elem:
        item.tau = 0.0
        print(item)        


   elem: 'nlinsert'
   name: 'NL1'
 zstart: 0.0
   zend: 1.8
  steps: 1000
   zlen: 1.8
      k: 1.4544633300000003
      c: 0.01
    tau: 0.0


In [15]:
M_impact,M_mli = compare(beam_impact,lattice_impact,elemList_impact,latticeElem_impact)

In [16]:
M_impact

,0,1,2,3,4,5
0,-0.309051,0.537734,1.771572e-08,1.277640e-07,1.525068e-21,0.251907
1,-1.682034,-0.309051,1.909945e-08,2.331634e-07,7.716732e-22,0.323682
2,0.000000,0.000000,-3.089949e-01,1.305113e+00,0.000000e+00,-0.000000
3,0.000000,0.000000,-6.930603e-01,-3.089949e-01,0.000000e+00,-0.000000
4,-11.660636,-9.074949,-8.971064e-07,5.279489e-06,1.000000e+00,-44.485230
5,-0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-4.454864e-13,1.000000


In [17]:
M_mli

,1,2,3,4,5,6
1,-0.309051,0.537734,0.000000,0.000000,0.0,0.251907
2,-1.682034,-0.309051,0.000000,0.000000,0.0,0.323682
3,0.000000,0.000000,-0.308995,1.305113,0.0,0.000000
4,0.000000,0.000000,-0.693060,-0.308995,0.0,0.000000
5,-0.323682,-0.251907,0.000000,0.000000,1.0,-1.234847
6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000


# from MLI Input 

### Sext off / NL on

In [18]:
elemList_mli,latticeList_mli,labor_mli = impact.MLI.readInputfile('../MLIchrom/MLIchromaticityNoSextWithInsert/mli.in')

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
tadm is not recognized. skipping...
tadm is not recognized. skipping...
sq is not recognized. skipping...
wsq is not recognized. skipping...
cod is not recognized. skipping...
cod is not recognized. skipping...


In [19]:
elemList_mli[2].driftexact = 1

In [20]:
beam_mli = impact.mli2impact.beam(elemList_mli)
lattice_mli = impact.mli2impact.lattice(elemList_mli,latticeList_mli[0],beam_mli)

monitor is not recognized. skipping...
vkicker is not recognized. replacing with drift...
MLI sextupole of length 0.1 found. Converting to 10 drift-kick-drift thin element
quadrupole strength not found in the following MLI element:

 elem: 'quadrupole'
 name: 'sqa1r'
    l: 0.1
  k1s: 0
monitor is not recognized. skipping...
monitor is not recognized. skipping...
quadrupole strength not found in the following MLI element:

 elem: 'quadrupole'
 name: 'sqa2r'
    l: 0.1
  k1s: 0
quadrupole strength not found in the following MLI element:

 elem: 'quadrupole'
 name: 'sqb1r'
    l: 0.1
  k1s: 0
monitor is not recognized. skipping...
marker is not recognized. skipping...
marker is not recognized. skipping...
monitor is not recognized. skipping...
quadrupole strength not found in the following MLI element:

 elem: 'quadrupole'
 name: 'sqb2r'
    l: 0.1
  k1s: 0
quadrupole strength not found in the following MLI element:

 elem: 'quadrupole'
 name: 'sqc1r'
    l: 0.1
  k1s: 0
monitor is not r

In [21]:
M_impact,M_mli = compare(beam_mli,lattice_mli,elemList_mli,latticeList_mli[0])

In [22]:
M_impact

,0,1,2,3,4,5
0,-0.699614,0.269225,9.899787e-10,8.775450e-08,0.0,0.327066
1,-1.280303,-0.936674,-4.808543e-10,1.393163e-07,0.0,0.246376
2,0.000000,0.000000,9.707311e-02,1.856125e+00,0.0,-0.000000
3,0.000000,0.000000,-4.742469e-01,1.233489e+00,0.0,-0.000000
4,-8.875658,-13.425926,3.819804e-08,4.772585e-05,1.0,-45.021191
5,-0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-0.0,1.000000


In [23]:
M_mli

,1,2,3,4,5,6
1,-0.699614,0.269226,0.000000,0.000000,0.0,0.327066
2,-1.280303,-0.936673,0.000000,0.000000,0.0,0.246376
3,0.000000,0.000000,0.097073,1.856125,0.0,0.000000
4,0.000000,0.000000,-0.474247,1.233490,0.0,0.000000
5,-0.246376,-0.372684,0.000000,0.000000,1.0,-1.249723
6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000


### Sext off / NL off

In [24]:
for item in lattice_mli:
    if 'insert' in item.type:
        item.strength_t = 0.0
        print(item)


               type: 'nonlinear_insert'
             length: 1.8 [m]
           n_sckick: 36 [1]
              n_map: 10 [1]
         strength_t: 0.0
 transverse_scale_c: 0.01
        pipe_radius: 1.0 [m]
       tune_advance: 0.30000000043929054


In [25]:
for item in elemList_mli:
    if 'insert' in item.elem:
        item.tau = 0.0
        print(item)        


   elem: 'nlinsert'
   name: 'insert1'
 zstart: 0.0
   zend: 1.8
  steps: 1000
   zlen: 1.8
      k: 1.45446333
      c: 0.01
    tau: 0.0


In [26]:
M_impact,M_mli = compare(beam_mli,lattice_mli,elemList_mli,latticeList_mli[0])

In [27]:
M_impact

,0,1,2,3,4,5
0,-0.309051,0.537734,1.743655e-09,1.938158e-08,0.0,0.251907
1,-1.682034,-0.309051,1.753438e-09,3.407523e-08,0.0,0.323682
2,0.000000,0.000000,-3.089949e-01,1.305113e+00,0.0,-0.000000
3,0.000000,0.000000,-6.930603e-01,-3.089949e-01,0.0,-0.000000
4,-11.660633,-9.074940,1.576378e-07,7.964733e-06,1.0,-44.485267
5,-0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-0.0,1.000000


In [28]:
M_mli

,1,2,3,4,5,6
1,-0.309051,0.537734,0.000000,0.000000,0.0,0.251907
2,-1.682034,-0.309051,0.000000,0.000000,0.0,0.323682
3,0.000000,0.000000,-0.308995,1.305113,0.0,0.000000
4,0.000000,0.000000,-0.693060,-0.308995,0.0,0.000000
5,-0.323682,-0.251907,0.000000,0.000000,1.0,-1.234847
6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
